In [1]:
from collections import Counter
import random

import numpy as np
import pyarrow.dataset as ds
import duckdb
import pandas as pd
import numpy as np
import seaborn as sns
import plotly as pl
import plotly.express as px
import json

from plotly.graph_objs import Figure

INDICADORES = {
    "AFD": "Adequação da Formação Docente",
    "ICG": "Complexidade de Gestão da Escola",
    "IED": "Esforço Docente",
    "ATU": "Média de Alunos por Turma",
    "HAD": "Média de Horas-aula diária",
    "DSU": "Percentual de Docentes com Curso Superior",
    "TDI": "Taxas de Distorção Idade-série"
}

DIMENSIONS = {
    "Ano": "NU_ANO_CENSO",
    "Nome da região geográfica": "NO_REGIAO",
    "Nome da Unidade da Federação": "NO_UF",
    "Nome da Mesorregião": "NO_MESORREGIAO",
    "Nome da Microrregião": "NO_MICRORREGIAO"
}

con = duckdb.connect('../file.db')

microdados = ds.dataset("../data/transformed/microdados.parquet", format="parquet", partitioning="hive")
con.register("microdados", microdados)

indicadores = {}
for indicador in INDICADORES:
    indicadores[indicador] = ds.dataset(f"../data/transformed/indicadores/{indicador}.parquet", format="parquet", partitioning="hive")
    con.register(indicador, indicadores[indicador])

In [5]:

niveis_ensino = [
    'Educação Básica', 'Educação Infantil - creche',
    'Educação Infantil - pré-escola', 'Educação Infantil',
    'Ensino Fundamental - anos iniciais',
    'Ensino Fundamental - anos finais',
    'Ensino Fundamental',
    # 'Ensino Médio', 'Educação Profissional',
    # 'Educação Profissional Técnica',
    # 'Educação de Jovens e Adultos (EJA) - Ensino Fundamental',
    # 'Educação de Jovens e Adultos (EJA) - Ensino Médio',
    # 'Educação de Jovens e Adultos (EJA)',
    # 'Educação Especial Inclusiva',
    # 'Educação Especial Exclusiva',
    'Educação Especial'
]

# Quantida de matriculas por nivel de ensino
quantidade_matriculas_nivel_ensino = con.execute("""
                select
                    NU_ANO_CENSO as 'Ano',
                    cast(sum(QT_MAT_BAS) as bigint) as 'Educação Básica',
                    cast(sum(QT_MAT_INF_CRE) as bigint) as 'Educação Infantil - creche',
                    cast(sum(QT_MAT_INF_PRE) as bigint) as 'Educação Infantil - pré-escola',
                    cast(sum(QT_MAT_INF) as bigint) as 'Educação Infantil',
                    cast(sum(QT_MAT_FUND_AI) as bigint) as 'Ensino Fundamental - anos iniciais',
                    cast(sum(QT_MAT_FUND_AF) as bigint) as 'Ensino Fundamental - anos finais',
                    cast(sum(QT_MAT_FUND) as bigint) as 'Ensino Fundamental',
                    cast(sum(QT_MAT_MED) as bigint) as 'Ensino Médio',
                    cast(sum(QT_MAT_PROF) as bigint) as 'Educação Profissional',
                    cast(sum(QT_MAT_PROF_TEC) as bigint) as 'Educação Profissional Técnica',
                    cast(sum(QT_MAT_EJA_FUND) as bigint) as 'Educação de Jovens e Adultos (EJA) - Ensino Fundamental',
                    cast(sum(QT_MAT_EJA_MED) as bigint) as 'Educação de Jovens e Adultos (EJA) - Ensino Médio',
                    cast(sum(QT_MAT_EJA) as bigint) as 'Educação de Jovens e Adultos (EJA)',
                    cast(sum(QT_MAT_ESP_CC) as bigint) as 'Educação Especial Inclusiva',
                    cast(sum(QT_MAT_ESP_CE) as bigint) as 'Educação Especial Exclusiva',
                    cast(sum(QT_MAT_ESP) as bigint) as 'Educação Especial'
                from microdados
                group by NU_ANO_CENSO
                order by 1
            """
                                                 ).df()
quantidade_matriculas_nivel_ensino = quantidade_matriculas_nivel_ensino.melt(
        id_vars='Ano',
        var_name="Nível de ensino",
        value_name="Quantidade de matrículas",
        value_vars=niveis_ensino
     )

px.line(
    quantidade_matriculas_nivel_ensino,
    x='Ano',
    y="Quantidade de matrículas",
    color="Nível de ensino",
    markers=True,
    title="Quantidade de matrículas por nível de ensino"
)

In [6]:
# quantidade de matriculas por dependencia
DIMENSOES_COMUNS = {
    "Dependência Administrativa": "TP_DEPENDENCIA",
    "Categoria de escola": "TP_CATEGORIA_ESCOLA_PRIVADA",
    "Localização": "TP_LOCALIZACAO",
    "Localização diferenciada da escola": "TP_LOCALIZACAO_DIFERENCIADA"
}

def get_plot_quantidade_matriculas(dimensao: str) -> (pl.graph_objs.Figure, pd.DataFrame):
    df = con.execute(f"""
                    select
                        NU_ANO_CENSO as 'Ano',
                        {DIMENSOES_COMUNS[dimensao]} as '{dimensao}',
                        count(*) as 'Quantidade de matrículas'
                    from microdados
                    group by NU_ANO_CENSO, {DIMENSOES_COMUNS[dimensao]}
                    order by 1, 2
                """
    ).df()
    fig = px.bar(
        df,
        x='Ano',
        y="Quantidade de matrículas",
        color=dimensao,
        title=f"Quantidade de matrículas por {dimensao.lower()}"
    )
    return fig, df

In [7]:
fig, df = get_plot_quantidade_matriculas("Dependência Administrativa")
fig

In [8]:
fig, df = get_plot_quantidade_matriculas("Categoria de escola")
fig

In [9]:
fig, df = get_plot_quantidade_matriculas("Localização")
fig

In [10]:
fig, df = get_plot_quantidade_matriculas("Localização diferenciada da escola")
fig

In [28]:
# Quantidade de matriculas por região geográfica
DIMENSOES_GEOGRAFICAS = {
    "Região geográfica": "NO_REGIAO",
    "Unidade da Federação": "NO_UF",
    "Mesorregião": "NO_MESORREGIAO",
    "Microrregião": "NO_MICRORREGIAO"
}

def get_plot_quantidade_matriculas_geografico(
        dimensao_geografica: str,
        nivel_ensino: str
) -> (pl.graph_objs.Figure, pd.DataFrame):
    df = con.execute(f"""
                    select
                        NU_ANO_CENSO as 'Ano',
                        {DIMENSOES_GEOGRAFICAS[dimensao_geografica]} as '{dimensao_geografica}',
                        cast(sum(QT_MAT_BAS) as bigint) as 'Educação Básica',
                        cast(sum(QT_MAT_INF_CRE) as bigint) as 'Educação Infantil - creche',
                        cast(sum(QT_MAT_INF_PRE) as bigint) as 'Educação Infantil - pré-escola',
                        cast(sum(QT_MAT_INF) as bigint) as 'Educação Infantil',
                        cast(sum(QT_MAT_FUND_AI) as bigint) as 'Ensino Fundamental - anos iniciais',
                        cast(sum(QT_MAT_FUND_AF) as bigint) as 'Ensino Fundamental - anos finais',
                        cast(sum(QT_MAT_FUND) as bigint) as 'Ensino Fundamental',
                        cast(sum(QT_MAT_MED) as bigint) as 'Ensino Médio',
                        cast(sum(QT_MAT_PROF) as bigint) as 'Educação Profissional',
                        cast(sum(QT_MAT_PROF_TEC) as bigint) as 'Educação Profissional Técnica',
                        cast(sum(QT_MAT_EJA_FUND) as bigint) as 'Educação de Jovens e Adultos (EJA) - Ensino Fundamental',
                        cast(sum(QT_MAT_EJA_MED) as bigint) as 'Educação de Jovens e Adultos (EJA) - Ensino Médio',
                        cast(sum(QT_MAT_EJA) as bigint) as 'Educação de Jovens e Adultos (EJA)',
                        cast(sum(QT_MAT_ESP_CC) as bigint) as 'Educação Especial Inclusiva',
                        cast(sum(QT_MAT_ESP_CE) as bigint) as 'Educação Especial Exclusiva',
                        cast(sum(QT_MAT_ESP) as bigint) as 'Educação Especial'
                    from microdados
                    group by NU_ANO_CENSO, {DIMENSOES_GEOGRAFICAS[dimensao_geografica]}
                    order by 1
                """
    ).df()

    fig = px.bar(
        df,
        x="Ano",
        y=nivel_ensino,
        color=dimensao_geografica,
        title=f"Quantidade de matrículas de {nivel_ensino} por {dimensao_geografica.lower()}",
        labels={nivel_ensino: "Quantidade de matrículas"}
    )

    return fig, df


In [29]:
fig, df = get_plot_quantidade_matriculas_geografico("Região geográfica", "Ensino Médio")
fig

In [51]:
def get_opcoes_geograficas(dimensao: str) -> list[str]:
    column = DIMENSOES_GEOGRAFICAS[dimensao]
    return list(np.sort(microdados.to_table(columns=[column]).to_pandas()[column].unique()))
get_opcoes_geograficas("Microrregião")

['Adamantina',
 'Afonso Cláudio',
 'Aglomeração Urbana de São Luís',
 'Agreste Potiguar',
 'Agreste de Itabaiana',
 'Agreste de Lagarto',
 'Aimorés',
 'Alagoana do Sertão do São Francisco',
 'Alagoinhas',
 'Alegre',
 'Alfenas',
 'Almeirim',
 'Almenara',
 'Alta Floresta',
 'Altamira',
 'Alto Araguaia',
 'Alto Capibaribe',
 'Alto Guaporé',
 'Alto Mearim e Grajaú',
 'Alto Médio Canindé',
 'Alto Médio Gurguéia',
 'Alto Pantanal',
 'Alto Paraguai',
 'Alto Parnaíba Piauiense',
 'Alto Solimões',
 'Alto Taquari',
 'Alto Teles Pires',
 "Alvorada D'Oeste",
 'Amapá',
 'Amparo',
 'Andradina',
 'Andrelândia',
 'Angicos',
 'Anicuns',
 'Anápolis',
 'Apucarana',
 'Aquidauana',
 'Aracaju',
 'Aragarças',
 'Araguaína',
 'Arapiraca',
 'Araranguá',
 'Araraquara',
 'Arari',
 'Araripina',
 'Araxá',
 'Araçatuba',
 'Araçuaí',
 'Arinos',
 'Aripuanã',
 'Ariquemes',
 'Assaí',
 'Assis',
 'Astorga',
 'Auriflama',
 'Avaré',
 'Bacia de São João',
 'Baixa Verde',
 'Baixada Maranhense',
 'Baixo Cotinguiba',
 'Baixo Cur

In [ ]:
microdados.to_table(columns=["NO_UF"]).to_pandas()["NO_UF"].unique()

In [35]:
microdados.

In [32]:
regiao_geografica = list(df["Região geográfica"].unique())

['Centro-Oeste', 'Norte', 'Nordeste', 'Sudeste', 'Sul']

In [149]:
#quantidade_matriculas_geo = quantidade_matriculas_geo[quantidade_matriculas_geo["Região geográfica"] == "Norte"]

In [150]:
px.bar(
    quantidade_matriculas_geo,
    x=dimension,
    y="Quantidade de matrículas",
    color="Nível de ensino",
)

In [34]:
# Quantidade de escolas
dimension = "Ano"
quantidade_escolas = con.execute(f"""
                select
                    {DIMENSIONS[dimension]} as '{dimension}',
                    count(CO_ENTIDADE) as 'Quantidade de escolas'
                from microdados
                group by {DIMENSIONS[dimension]}
                order by 1
            """
                                 ).df()
px.bar(
        quantidade_escolas,
        x=dimension,
        y='Quantidade de escolas',
        title=f"QUANTIDADE DE ESCOLAS X {dimension.upper()}"
    )

In [2]:
with open("../geojs-100-mun.json") as f:
    geojson = json.load(f)

In [115]:
# agua
df = con.execute(f"""
                select
                    NU_ANO_CENSO as 'Ano',
                    NO_REGIAO as 'Região',
                    round(cast(count(*) filter (where IN_AGUA_REDE_PUBLICA) as float) / count(*), 3) as 'Rede Pública',
                    round(cast(count(*) filter (where IN_AGUA_POCO_ARTESIANO) as float) / count(*), 3) as 'Poço artesiano',
                    round(cast(count(*) filter (where IN_AGUA_CACIMBA) as float) / count(*), 3) as 'Cacimba/Cisterna/Poço',
                    round(cast(count(*) filter (where IN_AGUA_FONTE_RIO) as float) / count(*), 3) as 'Fonte/Rio/Igarapé/Riacho/Córrego',
                    round(cast(count(*) filter (where IN_AGUA_INEXISTENTE) as float) / count(*), 3) as 'Não há abastecimento de água',
                    round(
                        cast(count(*) filter
                            (where not (IN_AGUA_REDE_PUBLICA or IN_AGUA_POCO_ARTESIANO or IN_AGUA_CACIMBA or IN_AGUA_FONTE_RIO or IN_AGUA_INEXISTENTE ))
                            as float)
                        / count(*)
                    , 3) as 'Sem informações'
                from microdados
                group by NU_ANO_CENSO, NO_REGIAO
                order by 1, 2
            """
).df()
df

,Ano,Região,Rede Pública,Poço artesiano,Cacimba/Cisterna/Poço,Fonte/Rio/Igarapé/Riacho/Córrego,Não há abastecimento de água,Sem informações
0,2016,Centro-Oeste,0.747,0.161,0.030,0.017,0.005,0.117
1,2016,Nordeste,0.461,0.120,0.128,0.021,0.052,0.263
2,2016,Norte,0.257,0.264,0.113,0.177,0.064,0.176
3,2016,Sudeste,0.715,0.056,0.031,0.021,0.002,0.205
4,2016,Sul,0.767,0.102,0.023,0.016,0.002,0.121
5,2017,Centro-Oeste,0.749,0.158,0.028,0.016,0.004,0.118
6,2017,Nordeste,0.458,0.122,0.127,0.019,0.042,0.279
7,2017,Norte,0.258,0.270,0.105,0.176,0.060,0.181
8,2017,Sudeste,0.734,0.048,0.028,0.017,0.002,0.198
9,2017,Sul,0.771,0.099,0.021,0.016,0.003,0.122


In [116]:
df = df.melt(
        id_vars=df.columns[:2],
        var_name="Serviço",
        value_name="Taxa de acesso",
        value_vars=df.columns[2:]
    )

df

,Ano,Região,Serviço,Taxa de acesso
0,2016,Centro-Oeste,Rede Pública,0.747
1,2016,Nordeste,Rede Pública,0.461
2,2016,Norte,Rede Pública,0.257
3,2016,Sudeste,Rede Pública,0.715
4,2016,Sul,Rede Pública,0.767
...,...,...,...,...
205,2022,Centro-Oeste,Sem informações,0.000
206,2022,Nordeste,Sem informações,0.000
207,2022,Norte,Sem informações,0.000
208,2022,Sudeste,Sem informações,0.000


In [117]:
df["dimensão | serviço"] = df.iloc[:, 1] + " | " + df.iloc[:, 2]

In [118]:
df

,Ano,Região,Serviço,Taxa de acesso,dimensão | serviço
0,2016,Centro-Oeste,Rede Pública,0.747,Centro-Oeste | Rede Pública
1,2016,Nordeste,Rede Pública,0.461,Nordeste | Rede Pública
2,2016,Norte,Rede Pública,0.257,Norte | Rede Pública
3,2016,Sudeste,Rede Pública,0.715,Sudeste | Rede Pública
4,2016,Sul,Rede Pública,0.767,Sul | Rede Pública
...,...,...,...,...,...
205,2022,Centro-Oeste,Sem informações,0.000,Centro-Oeste | Sem informações
206,2022,Nordeste,Sem informações,0.000,Nordeste | Sem informações
207,2022,Norte,Sem informações,0.000,Norte | Sem informações
208,2022,Sudeste,Sem informações,0.000,Sudeste | Sem informações


In [90]:
# energia
df = con.execute(f"""
                select
                    NU_ANO_CENSO as 'Ano',
                    NO_REGIAO as 'Região',
                    round(cast(count(*) filter (where IN_ENERGIA_REDE_PUBLICA) as float) / count(*), 3) as 'Rede Pública',
                    round(cast(count(*) filter (where IN_ENERGIA_GERADOR_FOSSIL) as float) / count(*), 3) as 'Gerador movido a combustível fóssil',
                    round(cast(count(*) filter (where IN_ENERGIA_RENOVAVEL) as float) / count(*), 3) as 'Fontes de energia renováveis ou alternativas',
                    round(cast(count(*) filter (where IN_ENERGIA_INEXISTENTE) as float) / count(*), 3) as 'Não há energia elétrica',
                    round(
                        cast(count(*) filter
                            (where not (IN_ENERGIA_REDE_PUBLICA or IN_ENERGIA_GERADOR_FOSSIL or IN_ENERGIA_RENOVAVEL or IN_ENERGIA_INEXISTENTE))
                            as float)
                        / count(*)
                    , 3) as 'Sem informações'
                from microdados
                group by NU_ANO_CENSO, NO_REGIAO
                order by 1, 2
            """
).df()
df

,Ano,Região,Rede Pública,Gerador movido a combustível fóssil,Fontes de energia renováveis ou alternativas,Não há energia elétrica,Sem informações
0,2016,Centro-Oeste,0.869,0.000,0.000,0.006,0.125
1,2016,Nordeste,0.718,0.000,0.000,0.014,0.268
2,2016,Norte,0.580,0.000,0.000,0.150,0.270
3,2016,Sudeste,0.794,0.000,0.000,0.001,0.206
4,2016,Sul,0.878,0.000,0.000,0.000,0.121
5,2017,Centro-Oeste,0.868,0.000,0.000,0.005,0.127
6,2017,Nordeste,0.706,0.000,0.000,0.011,0.283
7,2017,Norte,0.588,0.000,0.000,0.137,0.274
8,2017,Sudeste,0.799,0.000,0.000,0.000,0.201
9,2017,Sul,0.876,0.000,0.000,0.001,0.123


In [109]:
df = df.melt(
    id_vars=df.columns[:2],
    var_name="Serviço",
    value_name="Taxa de acesso",
    value_vars=df.columns[2:]
)

/tmp/ipykernel_13485/1262673022.py:1: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  df = df.melt(


In [ ]:
df["dimensão | serviço"] = df["Região"] + " | " + df

In [91]:
# esgoto
df = con.execute(f"""
                select
                    NU_ANO_CENSO as 'Ano',
                    NO_REGIAO as 'Região',
                    round(cast(count(*) filter (where IN_ESGOTO_REDE_PUBLICA) as float) / count(*), 3) as 'Rede Pública',
                    round(cast(count(*) filter (where IN_ESGOTO_FOSSA_SEPTICA) as float) / count(*), 3) as 'Fossa Séptica',
                    round(cast(count(*) filter (where IN_ESGOTO_FOSSA_COMUM) as float) / count(*), 3) as 'Fossa rudimentar/comum',
                    round(cast(count(*) filter (where IN_ESGOTO_FOSSA) as float) / count(*), 3) as 'Fossa',
                    round(cast(count(*) filter (where IN_ESGOTO_INEXISTENTE) as float) / count(*), 3) as 'Não há esgotamento sanitário',
                    round(
                        cast(count(*) filter
                            (where not (IN_ESGOTO_REDE_PUBLICA or IN_ESGOTO_FOSSA_SEPTICA or IN_ESGOTO_FOSSA_COMUM or IN_ESGOTO_FOSSA or IN_ESGOTO_INEXISTENTE ))
                            as float)
                        / count(*)
                    , 3) as 'Sem informações'
                from microdados
                group by NU_ANO_CENSO, NO_REGIAO
                order by 1, 2
            """
).df()
df


,Ano,Região,Rede Pública,Fossa Séptica,Fossa rudimentar/comum,Fossa,Não há esgotamento sanitário,Sem informações
0,2016,Centro-Oeste,0.398,0.000,0.000,0.499,0.010,0.117
1,2016,Nordeste,0.207,0.000,0.000,0.507,0.038,0.263
2,2016,Norte,0.069,0.000,0.000,0.591,0.176,0.176
3,2016,Sudeste,0.689,0.000,0.000,0.108,0.003,0.205
4,2016,Sul,0.494,0.000,0.000,0.420,0.003,0.121
5,2017,Centro-Oeste,0.402,0.000,0.000,0.493,0.008,0.118
6,2017,Nordeste,0.209,0.000,0.000,0.495,0.034,0.279
7,2017,Norte,0.072,0.000,0.000,0.582,0.177,0.181
8,2017,Sudeste,0.688,0.000,0.000,0.116,0.003,0.198
9,2017,Sul,0.498,0.000,0.000,0.415,0.004,0.122


In [94]:
# esgoto
df = con.execute(f"""
                select
                    NU_ANO_CENSO as 'Ano',
                    NO_REGIAO as 'Região',
                    round(cast(count(*) filter (where IN_LIXO_SERVICO_COLETA) as float) / count(*), 3) as 'Servico de coleta',
                    round(cast(count(*) filter (where IN_LIXO_QUEIMA) as float) / count(*), 3) as 'Queima',
                    round(cast(count(*) filter (where IN_LIXO_ENTERRA) as float) / count(*), 3) as 'Enterra',
                    round(
                            cast(count(*) filter (where IN_LIXO_DESTINO_FINAL_PUBLICO) as float)
                            / count(*),
                        3)
                        as 'Leva a uma destinação final financiada pelo poder público',
                    round(
                            cast(count(*) filter (where IN_LIXO_DESCARTA_OUTRA_AREA) as float)
                            / count(*),
                        3)
                        as 'Destinação do lixo - Descarta em outra área',
                    round(
                        cast(count(*) filter
                            (where not (IN_LIXO_SERVICO_COLETA or IN_LIXO_QUEIMA or IN_LIXO_ENTERRA or IN_LIXO_DESTINO_FINAL_PUBLICO or IN_LIXO_DESCARTA_OUTRA_AREA ))
                            as float)
                        / count(*)
                    , 3) as 'Sem informações'
                from microdados
                group by NU_ANO_CENSO, NO_REGIAO
                order by 1, 2
            """
).df()
df

,Ano,Região,Servico de coleta,Queima,Enterra,Leva a uma destinação final financiada pelo poder público,Destinação do lixo - Descarta em outra área,Sem informações
0,2016,Centro-Oeste,0.798,0.082,0.024,0.000,0.000,0.129
1,2016,Nordeste,0.454,0.260,0.009,0.000,0.000,0.297
2,2016,Norte,0.358,0.444,0.049,0.000,0.000,0.222
3,2016,Sudeste,0.750,0.048,0.002,0.000,0.000,0.209
4,2016,Sul,0.842,0.049,0.038,0.000,0.000,0.126
5,2017,Centro-Oeste,0.800,0.079,0.023,0.000,0.000,0.130
6,2017,Nordeste,0.460,0.242,0.009,0.000,0.000,0.308
7,2017,Norte,0.363,0.429,0.047,0.000,0.000,0.228
8,2017,Sudeste,0.760,0.045,0.002,0.000,0.000,0.202
9,2017,Sul,0.844,0.044,0.035,0.000,0.000,0.128


In [95]:
df.values[:, 2:].sum(axis=1)

array([1.032999973744154, 1.0199999772012234, 1.0730000175535679,
       1.0090000068303198, 1.0550000071525574, 1.0320000108331442,
       1.0190000049769878, 1.0669999942183495, 1.0089999993797392,
       1.0509999878704548, 1.0290000010281801, 1.0170000037178397,
       1.068999983370304, 1.0080000234302133, 1.045000022277236,
       1.0440000258386135, 1.0429999893531203, 1.1690000025555491,
       1.0149999763816595, 1.0560000012628734, 1.0329999858513474,
       1.0489999987185001, 1.1900000059977174, 1.0220000077970326,
       1.0550000197254121, 1.0339999962598085, 1.0519999992102385,
       1.1939999694004655, 1.0320000173524022, 1.0530000193975866,
       1.6130000203847885, 2.046999990940094, 1.8170000165700912,
       1.84299997985363, 1.4850000143051147], dtype=object)

In [97]:
# internet
# esgoto
df = con.execute(f"""
                select
                    NU_ANO_CENSO as 'Ano',
                    NO_REGIAO as 'Região',
                    round(cast(count(*) filter (where IN_INTERNET) as float) / count(*), 3) as 'Acesso a internet',
--                     round(cast(count(*) filter (where IN_INTERNET_ALUNOS) as float) / count(*), 3) as 'Para uso do alunos',
--                     round(cast(count(*) filter (where IN_INTERNET_ADMINISTRATIVO) as float) / count(*), 3) as 'Para uso adiministrativo',
--                     round(
--                         cast(count(*) filter (where IN_INTERNET_APRENDIZAGEM) as float)
--                         / count(*),
--                         3)
--                         as 'Para uso nos processos de ensino e aprendizagem',
--                     round(cast(count(*) filter (where IN_INTERNET_COMUNIDADE) as float) / count(*), 3) as 'Para uso da comunidade'
                from microdados
                group by NU_ANO_CENSO, NO_REGIAO
                order by 1, 2
            """
                 ).df()
df

,Ano,Região,Acesso a internet
0,2016,Centro-Oeste,0.795
1,2016,Nordeste,0.380
2,2016,Norte,0.307
3,2016,Sudeste,0.687
4,2016,Sul,0.802
5,2017,Centro-Oeste,0.793
6,2017,Nordeste,0.390
7,2017,Norte,0.308
8,2017,Sudeste,0.637
9,2017,Sul,0.809
